In [ ]:
import pandas as pd
df=pd.read_csv('../data/label_reviews.csv')

In [21]:
df["predicted_label"].unique()

array(['Valid', 'Irrelevant'], dtype=object)

In [22]:
tag_mapping_dict = {
    'Valid': 0,
    'Advertisement': 1,
    'Irrelevant': 2  ,
    'Rant_Without_Visit': 3,
}

df['label'] = df['predicted_label'].map(tag_mapping_dict)
df[df['label']==None]

,business_name,text,predicted_label,prediction_reason,label


In [ ]:
df_filtered = df.loc[:,['text','label']]
df_filtered=df_filtered.head(10)

In [25]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [35]:
train_df, temp_df = train_test_split(df_filtered, test_size=0.3, random_state=42) 

val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42) 

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

my_dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})
my_dataset_dict = my_dataset_dict.remove_columns('__index_level_0__')
print(my_dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 70
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 15
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 15
    })
})


In [36]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"],truncation=True)


tokenized_datasets = my_dataset_dict.map(tokenize_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

KeyboardInterrupt: 